In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
dataset = pd.read_csv("dados/historicoMaterias.csv")

In [3]:
dataset

,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia
...,...,...,...,...,...,...
10104,2018-12-22,https://web.archive.org/web/20181222014135/htt...,https://g1.globo.com/economia/tecnologia/notic...,Brasil ganha 10 milhões de internautas em 1 an...,"Em apenas 1 ano, o número de internautas no B...",tecnologia
10105,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Como a Coreia do Norte quer usar alta tecnolog...,A Coreia do Norte costuma ostentar seu aparat...,tecnologia
10106,2018-12-27,https://web.archive.org/web/20181227111959/htt...,https://g1.globo.com/economia/tecnologia/notic...,Venda de computadores desacelera por eleições ...,As vendas de computadores no Brasil desaceler...,tecnologia
10107,2018-12-28,https://web.archive.org/web/20181228065113/htt...,https://g1.globo.com/economia/tecnologia/notic...,"Instagram passa a mostrar posts na horizontal,...",O Instagram liberou na tarde desta quinta-fei...,tecnologia


Dropando colunas desncessárias para o objetivo da rede neural

In [4]:
dataset = dataset.drop(["url_noticia", "url_noticia_curto", "assunto", "data"], axis=1)

In [5]:
dataset

,titulo,conteudo_noticia
0,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do..."
1,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...
2,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T..."
3,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...
4,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...
...,...,...
10104,Brasil ganha 10 milhões de internautas em 1 an...,"Em apenas 1 ano, o número de internautas no B..."
10105,Como a Coreia do Norte quer usar alta tecnolog...,A Coreia do Norte costuma ostentar seu aparat...
10106,Venda de computadores desacelera por eleições ...,As vendas de computadores no Brasil desaceler...
10107,"Instagram passa a mostrar posts na horizontal,...",O Instagram liberou na tarde desta quinta-fei...


Como é possível observar na tabela acima, o banco de dados dos textos, ainda possuem alguns caracteres restritos, como /n, /t, /xa0 e etc. Neste caso, estarei tratando para que estes sejam removidos todos do banco de dados

In [6]:
x = 0

for string in dataset["conteudo_noticia"]:
    string = str(string).replace("\n", " ")
    string = str(string).replace("\t", " ")
    string = str(string).replace("\xa0", " ")
    dataset.iloc[x]["conteudo_noticia"] = string
    x+=1


Encurtamento prático

In [7]:
dataset = dataset.iloc[1:500]

In [8]:
dataset

,titulo,conteudo_noticia
1,"Para Dilma, é 'apressada' a tese de que emerge...",A presidente Dilma Rousseff afirmou nesta se...
2,"Temos sido capazes de reduzir a inflação', diz...","O presidente do Banco Central, Alexandre Tom..."
3,Governo argentino autoriza compra de dólares a...,O governo argentino anunciou nesta sexta-fei...
4,Petrobras desiste de parte da área de Bem-te-v...,A Petrobras afirmou que propôs à Agência Nac...
5,"Consórcio deverá investir R$ 5,1 bilhões em li...",Vencedor do leilão da linha de transmissão d...
...,...,...
495,"Põe na conta: Neymar faz golaço,dá drible na m...",Neymar fez o gol 190 de sua carreira na vitóri...
496,"Daniel Alves comemora gol, mas afirma: ''Resul...",A goleada brasileira por 4 a 0 no amistoso dia...
497,Amistosos de Argentina e Itália são destaque n...,A preparação dos países que vão disputar a C...
498,"‘Herói’ do México, jogador dos EUA admite cobr...","Aos 27 anos, o meia-atacante Graham Zusi deu u..."


Para que a tokenization seja efetiva, preciso tokenizar o texto todo existente no meu dataset. Este banco de dados possuí blocos de texto, ao invés de ser um texto corrido completo, neste caso, é por esta razão que eu somo o texto todo para então passar o tokenizador. Se eu passar ele individualmente em cada bloco de texto, cada linha do meu dataset teria uma tokenização diferente, ou seja, a mesma palavra em diferentes blocos de texto teria 2 tokens

In [9]:
corpus = ''

for string in dataset["conteudo_noticia"]:
    corpus += string

In [10]:
for string in dataset["titulo"]:
    corpus += string

In [11]:
corpus = corpus.lower().split("\n")

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

Quantidade total de palavras detectadas no texto

In [13]:
len(tokenizer.word_index)

19567

Nesta parte, eu pego o dataset original, e vou converte-lo para tokens

In [14]:
for x in range(len(dataset["conteudo_noticia"])):
    dataset["conteudo_noticia"].iloc[x] = tokenizer.texts_to_sequences([dataset["conteudo_noticia"].iloc[x]])[0]

C:\Users\migue\AppData\Local\Temp\ipykernel_11348\1525063593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["conteudo_noticia"].iloc[x] = tokenizer.texts_to_sequences([dataset["conteudo_noticia"].iloc[x]])[0]


In [15]:
for x in range(len(dataset["titulo"])):
    dataset["titulo"].iloc[x] = tokenizer.texts_to_sequences([dataset["titulo"].iloc[x]])[0]

C:\Users\migue\AppData\Local\Temp\ipykernel_11348\2710351441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["titulo"].iloc[x] = tokenizer.texts_to_sequences([dataset["titulo"].iloc[x]])[0]


In [16]:
dataset

,titulo,conteudo_noticia
1,"[10, 736, 17, 18893, 3, 2672, 2, 6, 2091, 1889...","[3, 117, 736, 1703, 151, 77, 156, 41, 444, 146..."
2,"[216, 294, 3629, 2, 2682, 3, 18896, 182, 2430,...","[1, 117, 5, 275, 408, 1221, 2430, 61, 77, 156,..."
3,"[225, 471, 18898, 1459, 2, 2435, 56, 348, 5, 50]","[1, 225, 471, 781, 77, 156, 41, 444, 6, 10506,..."
4,"[929, 5735, 2, 148, 9, 192, 2, 87, 1093, 1284,...","[3, 929, 151, 6, 4175, 47, 895, 264, 5, 1940, ..."
5,"[3640, 1049, 2699, 74, 102, 38, 186, 7, 896, 2...","[1286, 5, 1960, 9, 896, 2, 738, 2, 807, 1817, ..."
...,...,...
495,"[6620, 12, 175, 345, 299, 3461, 567, 5391, 12,...","[345, 99, 1, 100, 3702, 2, 44, 266, 12, 114, 1..."
496,"[657, 1196, 3548, 100, 21, 697, 19560, 258, 17...","[3, 1266, 149, 18, 131, 3, 63, 8, 793, 360, 5,..."
497,"[4095, 2, 556, 4, 640, 46, 581, 8, 460, 77, 188]","[3, 1033, 25, 627, 6, 445, 580, 3, 55, 5, 60, ..."
498,"[5, 1492, 86, 25, 353, 2325, 844, 19563]","[48, 615, 59, 1, 336, 137, 18859, 5928, 204, 1..."


In [17]:
maxTitle = max(len(tokens) for tokens in dataset["titulo"])
maxContent = max(len(tokens) for tokens in dataset["conteudo_noticia"])

In [18]:
for titulo in dataset["titulo"]:
    if len(titulo) > 0:
        for x in range(maxTitle - len(titulo)): 
            titulo.append(0)

In [19]:
for conteudo in dataset["conteudo_noticia"]:
    if len(conteudo) > 0:
        for x in range(maxContent - len(conteudo)): 
            conteudo.append(0)

In [20]:
dataset

,titulo,conteudo_noticia
1,"[10, 736, 17, 18893, 3, 2672, 2, 6, 2091, 1889...","[3, 117, 736, 1703, 151, 77, 156, 41, 444, 146..."
2,"[216, 294, 3629, 2, 2682, 3, 18896, 182, 2430,...","[1, 117, 5, 275, 408, 1221, 2430, 61, 77, 156,..."
3,"[225, 471, 18898, 1459, 2, 2435, 56, 348, 5, 5...","[1, 225, 471, 781, 77, 156, 41, 444, 6, 10506,..."
4,"[929, 5735, 2, 148, 9, 192, 2, 87, 1093, 1284,...","[3, 929, 151, 6, 4175, 47, 895, 264, 5, 1940, ..."
5,"[3640, 1049, 2699, 74, 102, 38, 186, 7, 896, 2...","[1286, 5, 1960, 9, 896, 2, 738, 2, 807, 1817, ..."
...,...,...
495,"[6620, 12, 175, 345, 299, 3461, 567, 5391, 12,...","[345, 99, 1, 100, 3702, 2, 44, 266, 12, 114, 1..."
496,"[657, 1196, 3548, 100, 21, 697, 19560, 258, 17...","[3, 1266, 149, 18, 131, 3, 63, 8, 793, 360, 5,..."
497,"[4095, 2, 556, 4, 640, 46, 581, 8, 460, 77, 18...","[3, 1033, 25, 627, 6, 445, 580, 3, 55, 5, 60, ..."
498,"[5, 1492, 86, 25, 353, 2325, 844, 19563, 0, 0,...","[48, 615, 59, 1, 336, 137, 18859, 5928, 204, 1..."


Escrevo o novo dataset após tokenizado, por questões de segurança

In [21]:
import csv 
with open('dados/historicoMateriasTratadoRed.csv', 'w', newline='') as csvfile:

    header = ["titulo", "conteudo_noticia"]
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset.iloc[x])

In [22]:
import csv 
with open('dados/tituloTratadoRed.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, delimiter=',')

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset["titulo"].iloc[x])

In [23]:
import csv 
with open('dados/conteudoTratadoRed.csv', 'w', newline='') as csvfile:

    writer = csv.writer(csvfile, delimiter=',')

    for x in range (len(dataset)):       
        # Verifico se existem células vazias, e se sim elas não são inseridas na versão tratada dos dados, pois
        # se fossem poderiam poluir a compreensão da rede neural
        if(len(dataset["conteudo_noticia"].iloc[x]) != 0 and len(dataset["titulo"].iloc[x]) != 0):           
            writer.writerow(dataset["conteudo_noticia"].iloc[x])

In [146]:
import json

with open("tokenizerRed.json", "w") as jsonfile:
    json.dump(tokenizer.to_json(), jsonfile)